In [1]:
import fusionsc as fsc
from fusionsc.devices import w7x
import numpy as np

In [2]:
config = w7x.standard().compute(w7x.defaultGrid())
await config

In [19]:
mapping = fsc.flt.computeMapping(
    config,
    
    mappingPlanes = [np.radians(36)],
    toroidalSymmetry = 1,
    
    r = np.linspace(4.5, 7, 200),
    z = np.linspace(-1.5, 1.5, 200),
    
    stepSize = 0.02,
    distanceLimit = 0,
    
    targetError = 1e-3, errorEstimationDistance = 1e4, minStepSize = 1e-3
)
await mapping

In [20]:
gm = mapping.mapGeometry(w7x.divertor())
geo = gm.getSection(0)

In [21]:
geo.rotate(np.radians(-90), [1,0,0]).asPyvista().plot(notebook=False)